## Cristiano Nicolau - 108536

## Problem
Write a program that implements a simple “People You Might Know” social 
network  friendship  recommendation  algorithm.  The  key  idea  is  that  if  two 
people have a lot of mutual friends, then the system should recommend that 
they connect with each other. 
 
Data: 
 
- Associated data file is soc-LiveJournal1Adj.txt found in the shared folder. 
- The file contains the adjacency list and has multiple lines in the following 
format: 
 
- <User ID> <Friend 1> <Friend 2> <Friend 3> ...

In [1]:
from pyspark.sql import SparkSession

In [2]:
sp = SparkSession.builder.appName("People You Might Know").getOrCreate()
sp.sparkContext.setLogLevel("ERROR")     

25/04/04 22:53:34 WARN Utils: Your hostname, cristianonicolau.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
25/04/04 22:53:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/04 22:53:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


A minha abordagem começa por ler o ficheiro txt e criar um dicionário onde a chave é o User ID e o valor é uma lista com os amigos desse User ID.

In [3]:
INPUT_PATH = "_input/soc-LiveJournal1Adj.txt"
OUTPUT_PATH = "_output/people-you-might-know"

In [4]:
def parse_line(line):
    """
    Esta funçao recebe uma linha do arquivo de entrada e retorna um par (user_id, friends)
    user_id é um inteiro unico e friends é uma lista de inteiros representando os amigos
    O que faço na função é dividir a linha em partes, onde a primeira parte é o user_id e após p tab temos a segunda parte que são os amigos
    """
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    friends = [int(friend) for friend in parts[1].split(',')] if len(parts) > 1 else []
    return user_id, friends

def load_network(file_path):
    """
    Esta função carrega a rede de amigos a partir do ficheiro
    Ela lê o arquivo, processa cada linha e retorna um dicionário onde as chaves são os user_ids e os valores são conjuntos de amigos.
    """
    network = {}
    try:
        rdd = sp.sparkContext.textFile(file_path)
        network_rdd = rdd.map(parse_line)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return {}
    except Exception as e:
        print(f"Error loading network: {str(e)}")
        return {}
    
    return network_rdd

In [5]:
network_rdd = load_network(INPUT_PATH)

print("Number of users:", network_rdd.count())
for user_id, friends in network_rdd.take(10):
    print(f"User {user_id} has {len(friends)} friends.")


Number of users: 49995
User 0 has 94 friends.
User 1 has 67 friends.
User 2 has 16 friends.
User 3 has 9 friends.
User 4 has 19 friends.
User 5 has 15 friends.
User 6 has 35 friends.
User 7 has 6 friends.
User 8 has 9 friends.
User 9 has 4 friends.


In [ ]:
def recommend_friends(network_rdd, N=10):
    pairs_rdd = network_rdd.flatMap(lambda x: [(x[0], friend) for friend in x[1]])
    
    mutual_friends_rdd = (
        pairs_rdd
        .map(lambda x: (x[1], x[0]))  
        .join(pairs_rdd.map(lambda x: (x[1], x[0])))  
        .filter(lambda x: x[1][0] < x[1][1]) 
        .map(lambda x: ((x[1][0], x[1][1]), 1)) 
        .reduceByKey(lambda a, b: a + b) 
    )
    
    existing_friendships = pairs_rdd.map(lambda x: (x[0], {x[1]})) \
                                   .reduceByKey(lambda a, b: a.union(b)) \
                                   .collectAsMap()
    
    recommendations_rdd = (
        mutual_friends_rdd
        .flatMap(lambda x: [
            (x[0][0], (x[0][1], x[1])),  
            (x[0][1], (x[0][0], x[1]))  
        ])
        .filter(lambda x: x[1][0] not in existing_friendships.get(x[0], set())) 
        .groupByKey()  
        .mapValues(lambda iter_friends: 
            ','.join(str(f[0]) for f in sorted(iter_friends, key=lambda x: (-x[1], x[0]))[:N])
        )
    )
    
    all_users = network_rdd.map(lambda x: x[0])
    users_with_recs = recommendations_rdd.map(lambda x: x[0])
    users_without_recs = all_users.subtract(users_with_recs).map(lambda user: (user, ""))
    
    final_recommendations = recommendations_rdd.union(users_without_recs)
    
    return final_recommendations

In [ ]:
recommendations_rdd = recommend_friends(network_rdd, N=10)
print("Number of recommendations:", recommendations_rdd.count())
for user_id, recs in recommendations_rdd.take(10):
    print(f"User {user_id} recommendations: {recs}")

Number of recommendations: 49995


User 68 recommendations: 1,2,3,4,5,6,7,8,9,10
User 27552 recommendations: 25186,38737,44070,19365,27600,10022,37448,18912,27086,27589
User 41444 recommendations: 27555,7785,27583,27585,27590,27679,38737,1489,9144,12633
User 25176 recommendations: 34278,28263,36683,3734,8685,20561,25225,29490,36706,36961
User 15712 recommendations: 439,1100,15114,15707,15730,16532,23305,26577,26603,26609
User 18320 recommendations: 9463,9770,9858,18298,18306,18576,20456,41782,52,503
User 49888 recommendations: 23512,13616,13722,34140,1923,23523,27536,8672,17636,20554
User 14044 recommendations: 13966,13867,14125,13911,13960,14123,14162,13917,14027,23660
User 13876 recommendations: 12519,8726,14073,7494,13867,13872,13877,13993,14026,14101
User 29772 recommendations: 29783,8721,43145,13430,19416,26883,28050,29780,29819,43143


In [ ]:
def save_recommendations(recommendations_rdd, output_path):
    formatted_recommendations = recommendations_rdd.map(
        lambda user_recs: f"{user_recs[0]}\t{user_recs[1]}"
    )
    formatted_recommendations.repartition(1).saveAsTextFile(output_path)
    print(f"Recommendations saved to {output_path}")

save_recommendations(recommendations_rdd, OUTPUT_PATH)

Recommendations saved to _output/people-you-might-know


In [10]:
sp.stop()

## Explicação do algoritmo

O algoritmo implementado é um sistema de recomendação de amizades baseado em amigos em comum. A ideia principal é que, se duas pessoas têm muitos amigos em comum, elas devem ser recomendadas uma para a outra.

1. **Carregamento da Rede de Amigos**:
    - O algoritmo começa por ler um ficheiro de entrada, onde cada linha representa um utilizador e os seus amigos.
        - Exemplo de linha: 1   2,3,4 (o utilizador 1 é amigo dos utilizadores 2, 3 e 4).

    - Cada linha é processada e convertida num par:
        - `(user_id, friends)`, onde:
            - `user_id`: número inteiro que representa o ID do utilizador.
            - `friends`: lista de inteiros com os IDs dos seus amigos diretos.

    - Os dados são carregados num RDD (Resilient Distributed Dataset) — estrutura Spark que permite fazer operações em paralelo, escalavel, tolerante a falhas e imutável.

2. **Pares de Amizades**:
    - Apos isso para cada utilizador, crio pares do tipo (utilizador, amigo) para representar as amizades existentes.
        - Exemplo: se o utilizador 1 tem amigos 2, 3 e 4, os pares seriam (1, 2), (1, 3) e (1, 4).
        - Isso é feito usando a função `flatMap`, que transforma cada lista de amigos num conjunto de pares (amigo, utilizador).
        - O resultado é uma lista de pares onde cada amigo é associado ao seu utilizador original.

    - Estes pares são depois reorganizados como (amigo, utilizador) para facilitar o processo seguinte, que consiste em encontrar utilizadores com amigos em comum.
        - Isso é feito usando a função `map`, que transforma cada par (utilizador, amigo) em (amigo, utilizador).
        - O resultado é uma lista de pares onde cada amigo é associado ao seu utilizador original.

    - Com esta reorganização consigo tratar os amigos como chaves e ver quem mais tem o mesmo amigo.

3. **Identificação de Amigos em Comum**:
    - DEpois o algoritmo junta os pares (amigo, utilizador) entre si — ou seja, encontra utilizadores diferentes que partilham o mesmo amigo.
        - Isto é feito usando a função `join`, que combina os pares (amigo, utilizador) com base no amigo em comum.
        - Exemplo: se tanto o A como o B são amigos do X, então encontramos o par (A, B) com um amigo em comum (X).

    - Para evitar repetições como (A, B) e (B, A), consideramos apenas os pares onde A < B.
        - Isso é feito usando a função `filter`, que remove os pares onde o utilizador A é maior ou igual ao utilizador B.
        - Exemplo: se temos os pares (1, 2) e (2, 1), mantemos apenas (1, 2). Com isto, garantimos que cada par de utilizadores é contado apenas uma vez.

    - No final, contamos quantas vezes cada par de utilizadores aparece com amigos em comum — ou seja, quantos amigos partilham.

4. **Filtragem de Amizades Existentes**:
    - Apos isso criamos um mapa com as amizades atuais (quem já é amigo de quem).
    - Antes de sugerir amizades novas, garantimos que nenhuma sugestão é de alguém que já é amigo do utilizador.
    
    
5. **Geração de Recomendações**:
    - A partir dos pares com amigos em comum (e que ainda não são amigos), o algoritmo gera sugestões.
    - Para cada utilizador:
        - Ordena-se a lista de potenciais amigos por:
            - Número de amigos em comum (ordem decrescente).
            - ID do utilizador recomendado (ordem crescente, para desempatar).
            - Selecionam-se os N melhores candidatos (por defeito, 10) e formatam-se como uma string.

6. **Utilizadores Sem Recomendações**:
    - Alguns utilizadores podem não ter amigos em comum com ninguém fora do seu círculo atual.
    - Nestes casos, o algoritmo garante que também são incluídos no resultado, mas com uma lista de recomendações vazia.

7. **Save das Recomendações**:
    - As recomendações finais são formatadas e guardadas num ficheiro de saída.
